In [12]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time

# Get data from website using scraping

In [21]:
URL = 'https://cycling.data.tfl.gov.uk/'
page = requests.get(URL)

#create bs4 object
soup = BeautifulSoup(page.content, "html.parser")

In [22]:
job_elements = soup.find_all("tr")

- Webpage uses JS to load the table, so we will use selenium first to scrape the content and then BS4 to get links for files - before then using HTTP requests for the content

In [27]:
# commented out - the below code initially saved the flat HTML content in a html file, which is saved in the data folder

# driver = webdriver.Edge()
# driver.get(URL)
# driver.set_window_position(0, 0)
# driver.set_window_size(100000, 200000)
# driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
# time.sleep(5) # wait to load

# # now print the response
# #print(driver.page_source)

# soup = BeautifulSoup(driver.page_source, "html.parser")

# with open("data\\tfl_data.html", "w") as file:
#     file.write(str(soup))

In [33]:
#open html file of the tfl website

with open('data\\tfl_data.html', 'r') as file:
    html_content = file.read()

In [35]:
#convert to BS4 object for scraping

soup = BeautifulSoup(html_content, "html.parser")

In [41]:
# get table with all files in

table = soup.find(id="tbody-content")

In [65]:
#get all URLs from the table for download

all_links = []

for tr in table.find_all('tr'):
    dl = tr.get('data-level')

    #only get level 3 links
    if dl == '3':
        # get a href tag for download link
        a_links = tr.find_all('a', href=True)
        # only get csv files
        if tr.find_all('td')[3].string == "CSV file":
            # only add link where there is a link that exists
            if len(a_links) > 0:
                all_links.append(a_links[0]['href'])

In [81]:
# get the links that are just for the usage stats that are for years 2019-2021

usage_links_all = []

for l in all_links:
    if l[32:43] == 'usage-stats' and (l[-6:-4] == '19' or l[-6:-4] == '20' or l[-6:-4] == '21'):
        usage_links_all.append(l)

In [83]:
#remove the first url which is data for 2018 to first day of 2019
usage_links_all = usage_links_all[1:]

In [90]:
# This is all commented to reduce run time.
# The below code requested all usage stats data and then saved the data in a csv file


# # to allow for csv reading
# storage_options = {'User-Agent': 'Mozilla/5.0'}

# def merge_csv_data(urls):
#     dfs = []
#     for url in urls:
#         # Read CSV data from URL
#         df = pd.read_csv(url, storage_options=storage_options)
#         # Append dataframe to list
#         dfs.append(df)
    
#     # Concatenate all dataframes in the list into one dataframe
#     merged_df = pd.concat(dfs, ignore_index=True)
    
#     return merged_df

# # List of URLs pointing to CSV files
# urls = usage_links_all

# # Call the function and get the merged dataframe
# merged_dataframe = merge_csv_data(urls)

# merged_dataframe.to_csv('data\\all_data.csv')

          Rental Id  Duration  Bike Id          End Date  EndStation Id  \
0          83284852       660     8282  02/01/2019 17:47             94   
1          83360769       180     4657  06/01/2019 18:14             94   
2          83280311       960     6306  02/01/2019 14:49            374   
3          83323626       120     6217  04/01/2019 12:54            269   
4          83343322       120      848  05/01/2019 16:03            269   
...             ...       ...      ...               ...            ...   
31348497  115876448      1080    19041  28/12/2021 13:14            148   
31348498  115859467      1440      203  26/12/2021 16:15              9   
31348499  115851371      1260    19576  25/12/2021 18:03            602   
31348500  115850742       900     9180  25/12/2021 17:23            794   
31348501  115860466       600    17062  26/12/2021 17:24            149   

                            EndStation Name        Start Date  \
0                 Bricklayers Arms

# Importing data from saved CSV file

In [93]:
df = pd.read_csv('data\\all_data.csv')

In [94]:
df.shape

(31348502, 10)

- Very large dataset, 31 million rows of data!

In [95]:
df.dtypes

Unnamed: 0            int64
Rental Id             int64
Duration              int64
Bike Id               int64
End Date             object
EndStation Id         int64
EndStation Name      object
Start Date           object
StartStation Id       int64
StartStation Name    object
dtype: object

In [100]:
#drop unnamed:0
df.drop(['Unnamed: 0'], axis=1, inplace=True)

KeyError: "['Unnamed: 0'] not found in axis"

In [99]:
# convert start and end dates to DateTime values
df['Start Date'] = pd.to_datetime(df['Start Date'], dayfirst=True)
df['End Date'] = pd.to_datetime(df['End Date'], dayfirst=True)